In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import h5py

import sys
sys.path.append("/Users/lsp/Documents/projets/en_cours/eeg/eeg")
from eeg import create3DMatrix, loadEEG, getEvents, getTrialNumList, plotFFTElectrodes, plot3DMatrix, computePickEnergy

sys.path.append("/Users/lsp/Documents/projets/en_cours/eeg/dss")
from dss import crossValidateDSS, createBiasedSignal

from tools.eeg_utils import processEEG
from tools.audio import getAttendedAndUnattendedEnv
from tools.behavior import getBehaviorData, analyses, checkLinkTrialsBehaviorEEG
from tools.decodingTRF import getTRFAccuracyByDur, getUnattendedTrialsNumber, calculateCorr
from tools.decodingSSR import calculateBaseline, comparePicks, getSSRAccuracyByDur, crossVal


In [ ]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
%matplotlib notebook

In [ ]:
# Plot parameters
plt.style.use('ggplot')
plt.rcParams['axes.facecolor']='w'

%pylab inline
pylab.rcParams['figure.figsize'] = (18, 10)

# Preprocessing

The goal of this notebook is to preprocess the raw data in order to create a file for each participant containing:

- The EEG data filtered between 1 and 15 Hz and downsampled to 64 Hz for the stimulus reconstruction (TRF). The shape of this matrix is (trial, time, electrode).
- The EEG data filtered between 1 and 100 Hz for the auditory steady-state response analysis (aSSR). The shape of this matrix is (trial, time, electrode).
- The envelopes of the attended stimuli. The shape of this matrix is (trial, time).
- The envelopes of the unattended stimuli. The shape of this matrix is (trial, time).

In [ ]:
def processingEEG(fnEEG, dbName, sessionNums, trialsToRemove, fs):
    password = "a"
    dbAddress = "https://db.auditory.fr:6984/"
    
    # Get behavior data
    trialBehavior = getBehaviorData(dbName, sessionNums)
    
    # Processing for TRF and SSR
    dataFilt3DTRF64, dataFilt3DSSR = processEEG(fnEEG, dbName, sessionNums,
                                                trialsToRemove, trialBehavior,
                                                fs, ref='average')
    # Get the envelopes for TRF
    envAttended, envUnattended = getAttendedAndUnattendedEnv(dbAddress, dbName,
                                                         password, verbose=True,
                                                            fs=48000.)

    # Get the minimum length to have all matrices the same length for TRF
    minLen = np.min([envAttended.shape[1], envUnattended.shape[1], dataFilt3DTRF64.shape[1]])
    
    dataFilt3DTRF64 = dataFilt3DTRF64[:, :minLen, :]
    envAttended = envAttended[:, :minLen]
    envUnattended = envUnattended[:, :minLen]
    
    # Export eeg data and attended envelope to do the TRF in Matlab and eeg for SSR (caching)
    with h5py.File('data_preproc/data_%s1.h5' % dbName, 'w') as hf:
        hf.create_dataset("eeg_TRF", data=dataFilt3DTRF64)
        hf.create_dataset("eeg_aSSR", data=dataFilt3DSSR)
        hf.create_dataset("envAttended", data=envAttended)
        hf.create_dataset("envUnattended", data=envUnattended)
    
    return dataFilt3DTRF64, envAttended, envUnattended, dataFilt3DSSR, trialBehavior

We used the function checkLinkTrialsBehaviorEEG (see package doc [here]()) to check the coherence between behavior and EEG. The triggers have to be updated:

Remove the following trials:

- hijee_18_06: 10
- thomas_20_06_18: nothing
- isaac_21_06: nothing
- thomas_s_22_06_18: nothing

# Preprocess data for each participant

In [ ]:
dataFilt3DTRF64_p1, envAttended_p1, envUnattended_p1, dataFilt3DSSR_p1, trialBehavior_p1 = processingEEG(fnEEG='data_raw/p1.bdf',
                                                      dbName="hijee_18_06",
                                                      sessionNums=np.arange(1, 9),
                                                      trialsToRemove=[10],
                                                      fs=512.)


In [ ]:
dataFilt3DTRF64_p2, envAttended_p2, envUnattended_p2, dataFilt3DSSR_p2, trialBehavior_p2 = processingEEG(fnEEG='data_raw/p2.bdf',
                                                               dbName="thomas_20_06",
                                                               sessionNums=np.arange(1, 9),
                                                               trialsToRemove=[],
                                                               fs=512.)

In [ ]:
dataFilt3DTRF64_p3, envAttended_p3, envUnattended_p3, dataFilt3DSSR_p3, trialBehavior_p3 = processingEEG(fnEEG='data_raw/p3.bdf',
                                                               dbName="isaac_21_06",
                                                               sessionNums=np.arange(1, 9),
                                                               trialsToRemove=[],
                                                               fs=512.)

In [ ]:
dataFilt3DTRF64_p4, envAttended_p4, envUnattended_p4, dataFilt3DSSR_p4, trialBehavior_p4 = processingEEG(fnEEG='data_raw/p4.bdf',
                                                               dbName="thomas_22_06",
                                                               sessionNums=np.arange(1, 9),
                                                               trialsToRemove=[],
                                                               fs=512.)